# Business Data Inspection

In [2]:
import random
random.seed(109)

from pprint import pprint

import os
import sys
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

## Business data

Contains business data including location data, attributes, and categories.

In [3]:
busi_df = pd.read_json('data/yelp_academic_dataset_business.json', 
                       orient = "records", lines = True)
print(f"business data shape: {busi_df.shape}")

business data shape: (150346, 14)


In [4]:
busi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [5]:
busi_df.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


## Data missingness

In [6]:
# Count the number of missing values for each column
busi_df.isnull().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

Only 3 columns have missing values: `attributes`, `categories`, and `hours`. 



### Business `categories` feature

The business data contains businesses that are not restaurants such as spas, hotels, and hair salons. We are interested only in restaurant recommendations. So we drop all rows whose categories do not contain any of the following keywords: "food", "restaurant", "bar", "pubs", "restaurant", "tea", "coffee". As a result, there are 69,253 out of 150,346 businesses left for analysis.

In [7]:
busi_df['categories'].sample(10)

20310                       Adult Entertainment, Nightlife
84895                        Food, Specialty Food, Grocery
9073                            Nail Salons, Beauty & Spas
97920                                   Pizza, Restaurants
86188     Health & Medical, Home Health Care, Optometrists
34203    Automotive, Car Dealers, Auto Repair, Auto Par...
10105    Shoe Stores, Men's Clothing, Fashion, Shopping...
14656                 Local Services, Sewing & Alterations
24730                  Bars, Nightlife, Pizza, Restaurants
96340          Afghan, Restaurants, Indian, Middle Eastern
Name: categories, dtype: object

In [8]:
busi_df['is_restaurant'] = busi_df['categories'].str.contains("food|bar|pubs|restaurant|tea|coffee", regex = True, case = False)
print(f"There are {busi_df['is_restaurant'].sum()} restaurants in the business dataset.")
# keep only businesses that are restaurants or food/beverage providers
busi_df = busi_df[busi_df['is_restaurant'] == True]

busi_df.head(3)

There are 69253 restaurants in the business dataset.


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,is_restaurant
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",True
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",True
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",True


### Business `attributes` features

The `attributes` feature contains an object (business attributes to values). Examples of `attributes` values:

In [9]:
# drop rows whose attributes are missing because they do not have enough information for restaurant recommendations
print(f"There are {busi_df['attributes'].isna().sum()} businesses with missing attribtues.")
busi_df = busi_df[~busi_df['attributes'].isna()]
print(f"There are {busi_df['is_restaurant'].sum()} restaurants in the business dataset.")

busi_df.reset_index(inplace = True, names = ['original_index'])
busi_df.head(3)

There are 967 businesses with missing attribtues.
There are 68286 restaurants in the business dataset.


,original_index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,is_restaurant
0,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",True
1,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",True
2,5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",True


In [10]:
pprint(list(busi_df.sample(3)['attributes']))

[{'BusinessAcceptsBitcoin': 'False',
  'BusinessAcceptsCreditCards': 'True',
  'BusinessParking': 'None',
  'Caters': 'True',
  'OutdoorSeating': 'False',
  'RestaurantsDelivery': 'False',
  'RestaurantsReservations': 'False',
  'RestaurantsTakeOut': 'True',
  'WiFi': "u'no'"},
 {'Alcohol': "u'full_bar'",
  'Ambience': "{u'divey': None, u'hipster': None, u'casual': True, "
              "u'touristy': None, u'trendy': True, u'intimate': None, "
              "u'romantic': False, u'classy': False, u'upscale': False}",
  'BestNights': "{u'monday': False, u'tuesday': False, u'wednesday': False, "
                "u'thursday': False, u'friday': False, u'saturday': False, "
                "u'sunday': False}",
  'BusinessParking': "{'garage': False, 'street': False, 'validated': False, "
                     "'lot': True, 'valet': False}",
  'CoatCheck': 'False',
  'HasTV': 'True',
  'Music': "{u'dj': False, u'live': True, u'jukebox': False, u'video': False, "
           "u'background_music'

Expanding the `attributes` feature into a flat table using `pd.json_normalize()` yields a table of 39 columns. 

In [11]:
busi_attr_df = pd.json_normalize(busi_df['attributes'])
print(f"attributes flat table shape: {busi_attr_df.shape}")
busi_attr_df.head(3)

attributes flat table shape: (68286, 39)


,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BusinessParking,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,Alcohol,...,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,AcceptsInsurance,RestaurantsCounterService,Open24Hours,AgesAllowed,DietaryRestrictions
0,False,False,False,"{'garage': False, 'street': True, 'validated':...",True,1,True,False,u'free',u'none',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,True,"{'garage': None, 'street': None, 'validated': ...",True,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,None,False,1,True,False,u'no',u'none',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We further expand business parking options to a flat table and concatenate the new columns to the previous `busi_attr_df`. However, the values of `BusinessParking` are enclosed as strings (unless they are missing values which would be floats). So, first we must convert them to `dict` and then expand the columns using `pd.json_normalize()`.


In [12]:
busi_attr_df.iloc[4]['BusinessParking']

"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"

In [13]:
# fill nan with empty dict string and convert values to dict type
busi_attr_df['BusinessParking'] = busi_attr_df['BusinessParking'].fillna("{}").apply(eval)

# expand and concatenate to the previous `busi_attr_df`
# busi_attr_df = pd.concat([busi_attr_df, 
#                           pd.json_normalize(busi_attr_df['BusinessParking']).add_prefix('parking_')],
#                          axis = 1)
busi_attr_df = busi_attr_df.merge(pd.json_normalize(busi_attr_df['BusinessParking']).add_prefix('parking_'),
                                  left_index = True, right_index = True, how = 'left')

# drop the original `BusinessParking` column
busi_attr_df.drop(columns = ['BusinessParking'], inplace = True)

busi_attr_df.head(3)

,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,Alcohol,Caters,...,AcceptsInsurance,RestaurantsCounterService,Open24Hours,AgesAllowed,DietaryRestrictions,parking_garage,parking_street,parking_validated,parking_lot,parking_valet
0,False,False,False,True,1,True,False,u'free',u'none',True,...,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
1,NaN,NaN,True,True,NaN,True,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,None,None,None,True,False
2,True,True,True,False,1,True,False,u'no',u'none',False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Similar to the parking data, `Ambience`, `DietaryRestrictions`, and `Music` features are expanded and concatenated to the attributes df.

In [14]:
# fill nan with empty dict string and convert values to dict type
busi_attr_df['Ambience'] = busi_attr_df['Ambience'].fillna("{}").apply(eval)

# expand and concatenate to the previous `busi_attr_df`
# busi_attr_df = pd.concat([busi_attr_df, 
#                           pd.json_normalize(busi_attr_df['Ambience']).add_prefix('ambience_')],
#                          axis = 1)
busi_attr_df = busi_attr_df.merge(pd.json_normalize(busi_attr_df['Ambience']).add_prefix('ambience_'),
                                  left_index = True, right_index = True, how = 'left')

# drop the original `Ambience` column
busi_attr_df.drop(columns = ['Ambience'], inplace = True)

busi_attr_df.head(3)

,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,Alcohol,Caters,...,parking_valet,ambience_romantic,ambience_intimate,ambience_touristy,ambience_hipster,ambience_divey,ambience_classy,ambience_trendy,ambience_upscale,ambience_casual
0,False,False,False,True,1,True,False,u'free',u'none',True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,True,True,NaN,True,NaN,NaN,NaN,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,False,1,True,False,u'no',u'none',False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# fill nan with empty dict string and convert values to dict type
busi_attr_df['DietaryRestrictions'] = busi_attr_df['DietaryRestrictions'].fillna("{}").apply(eval)

# expand and concatenate to the previous `busi_attr_df`
# busi_attr_df = pd.concat([busi_attr_df, 
#                           pd.json_normalize(busi_attr_df['DietaryRestrictions']).add_prefix('diet_')],
#                          axis = 1)
busi_attr_df = busi_attr_df.merge(pd.json_normalize(busi_attr_df['DietaryRestrictions']).add_prefix('diet_'),
                                  left_index = True, right_index = True, how = 'left')

# drop the original `DietaryRestrictions` column
busi_attr_df.drop(columns = ['DietaryRestrictions'], inplace = True)

busi_attr_df.head(3)

,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,Alcohol,Caters,...,ambience_trendy,ambience_upscale,ambience_casual,diet_dairy-free,diet_gluten-free,diet_vegan,diet_kosher,diet_halal,diet_soy-free,diet_vegetarian
0,False,False,False,True,1,True,False,u'free',u'none',True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,True,True,NaN,True,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,False,1,True,False,u'no',u'none',False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# fill nan with empty dict string and convert values to dict type
busi_attr_df['Music'] = busi_attr_df['Music'].fillna("{}").apply(eval)

# expand and concatenate to the previous `busi_attr_df`
# busi_attr_df = pd.concat([busi_attr_df, 
#                           pd.json_normalize(busi_attr_df['Music']).add_prefix('music_')],
#                          axis = 1)
busi_attr_df = busi_attr_df.merge(pd.json_normalize(busi_attr_df['Music']).add_prefix('music_'),
                                  left_index = True, right_index = True, how = 'left')

# drop the original `Music` column
busi_attr_df.drop(columns = ['Music'], inplace = True)

busi_attr_df.head(3)

,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,Alcohol,Caters,...,diet_halal,diet_soy-free,diet_vegetarian,music_dj,music_background_music,music_no_music,music_jukebox,music_live,music_video,music_karaoke
0,False,False,False,True,1,True,False,u'free',u'none',True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,True,True,NaN,True,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,False,1,True,False,u'no',u'none',False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check the number of missing values in these expanded attributes features:

In [17]:
print(dict(busi_attr_df.isna().sum()))

{'RestaurantsDelivery': 14369, 'OutdoorSeating': 20258, 'BusinessAcceptsCreditCards': 8068, 'BikeParking': 20733, 'RestaurantsPriceRange2': 10181, 'RestaurantsTakeOut': 10333, 'ByAppointmentOnly': 61228, 'WiFi': 24856, 'Alcohol': 25625, 'Caters': 28415, 'WheelchairAccessible': 50348, 'GoodForKids': 24945, 'RestaurantsAttire': 29114, 'RestaurantsReservations': 23303, 'CoatCheck': 63054, 'DogsAllowed': 54521, 'RestaurantsTableService': 48426, 'RestaurantsGoodForGroups': 24634, 'HasTV': 23921, 'HappyHour': 53530, 'DriveThru': 60610, 'NoiseLevel': 30924, 'GoodForMeal': 39207, 'BusinessAcceptsBitcoin': 59712, 'Smoking': 64014, 'GoodForDancing': 63807, 'BestNights': 62752, 'BYOB': 63836, 'Corkage': 64735, 'BYOBCorkage': 66842, 'HairSpecializesIn': 68060, 'AcceptsInsurance': 68184, 'RestaurantsCounterService': 68267, 'Open24Hours': 68260, 'AgesAllowed': 68165, 'parking_garage': 10811, 'parking_street': 12185, 'parking_validated': 11027, 'parking_lot': 11160, 'parking_valet': 8914, 'ambience_r

Group values of some `attributes` entries:

In [18]:
print("Before grouping `Alcohol`:")
print("\t", list(busi_attr_df['Alcohol'].unique()))

busi_attr_df['Alcohol'] = busi_attr_df['Alcohol'].map({"u'none'": "None",
                                                "'none'": "None",
                                                'None': "None",
                                                "u'full_bar'": "full_bar",
                                                "'full_bar'": "full_bar",
                                                "u'beer_and_wine'": "beer&wine",
                                                "'beer_and_wine'": "beer&wine"
                                                })
print("After grouping `Alcohol`:")
print("\t", list(busi_attr_df['Alcohol'].unique()))

Before grouping `Alcohol`:
	 ["u'none'", nan, "u'full_bar'", "'none'", "'full_bar'", "u'beer_and_wine'", "'beer_and_wine'", 'None']
After grouping `Alcohol`:
	 ['None', nan, 'full_bar', 'beer&wine']


In [19]:
print("Before grouping `WiFi`:")
print("\t", list(busi_attr_df['WiFi'].unique()))
busi_attr_df['WiFi'] = busi_attr_df['WiFi'].map({"u'no'": "no",
                                                "'no'": "no",
                                                'None': "None",
                                                "u'free'": "free",
                                                "u'paid'": "paid",
                                                "'paid'": "paid"})

print("After grouping `WiFi`:")
print("\t", list(busi_attr_df['WiFi'].unique()))

Before grouping `WiFi`:
	 ["u'free'", nan, "u'no'", "'free'", "'no'", "u'paid'", "'paid'", 'None']
After grouping `WiFi`:
	 ['free', nan, 'no', 'paid', 'None']


In [20]:
print("Before grouping `RestaurantsAttire`:")
print("\t", list(busi_attr_df['RestaurantsAttire'].unique()))
busi_attr_df['RestaurantsAttire'] = busi_attr_df['RestaurantsAttire'].map({"u'casual'": "casual",
                                                "'casual'": "casual",
                                                "u'formal'": "formal",
                                                "'dressy'": "formal",
                                                "u'dressy'": "formal",
                                                "'formal'": "formal",
                                                "None": "None"})

# fill na with "casual" since it is the most likely for restaurants if not indicated
busi_attr_df['RestaurantsAttire'].fillna("casual", inplace = True)

print("After grouping `RestaurantsAttire`:")
print("\t", list(busi_attr_df['RestaurantsAttire'].unique()))

Before grouping `RestaurantsAttire`:
	 [nan, "u'casual'", "'casual'", "u'formal'", "'dressy'", "u'dressy'", "'formal'", 'None']
After grouping `RestaurantsAttire`:
	 ['casual', 'formal', 'None']


In [21]:
# further drop rows that are not restaurants (e.g. `HairSpecializesIn` is not missing)
busi_attr_df = busi_attr_df[busi_attr_df['HairSpecializesIn'].isna()]
busi_attr_df.drop(columns = ['HairSpecializesIn'], inplace = True)
busi_attr_df.head()

,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,Alcohol,Caters,...,diet_halal,diet_soy-free,diet_vegetarian,music_dj,music_background_music,music_no_music,music_jukebox,music_live,music_video,music_karaoke
0,False,False,False,True,1,True,False,free,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,True,True,NaN,True,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,False,1,True,False,no,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,True,True,NaN,1,True,NaN,free,full_bar,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,True,True,NaN,1,True,False,no,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Imputing missing values for `attributes` features

The missing values among the business `attributes` features are filled with "None". The reason for this choice of missingness imputation is that users are more likely to be indifferent of these restaurants' characteristics if they do not specify it on the Yelp website. Therefore, the missing values have its own category "None".

In [22]:
busi_attr_df.fillna("None", inplace = True)
busi_attr_df

,RestaurantsDelivery,OutdoorSeating,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,RestaurantsTakeOut,ByAppointmentOnly,WiFi,Alcohol,Caters,...,diet_halal,diet_soy-free,diet_vegetarian,music_dj,music_background_music,music_no_music,music_jukebox,music_live,music_video,music_karaoke
0,False,False,False,True,1,True,False,free,None,True,...,None,None,None,None,None,None,None,None,None,None
1,None,None,True,True,None,True,None,None,None,False,...,None,None,None,None,None,None,None,None,None,None
2,True,True,True,False,1,True,False,no,None,False,...,None,None,None,None,None,None,None,None,None,None
3,False,True,True,None,1,True,None,free,full_bar,True,...,None,None,None,None,None,None,None,None,None,None
4,True,True,True,None,1,True,False,no,None,False,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68281,False,True,True,True,1,False,None,None,None,False,...,None,None,None,None,None,None,None,None,None,None
68282,False,True,True,True,2,True,None,None,None,False,...,None,None,None,None,None,None,None,None,None,None
68283,True,False,True,True,2,True,None,free,None,True,...,None,None,None,None,None,None,None,None,None,None
68284,False,None,True,True,1,True,None,None,None,True,...,None,None,None,None,None,None,None,None,None,None


In [23]:
# concatenate attributes features to the original business data
busi_df2 = busi_df.merge(busi_attr_df.astype(str), how = "inner", left_index = True, right_index = True)
print(f"There are {busi_df2.shape[0]} business records left with {busi_df2.shape[1]} total features.")
busi_df2.head()

There are 68060 business records left with 78 total features.


,original_index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,...,diet_halal,diet_soy-free,diet_vegetarian,music_dj,music_background_music,music_no_music,music_jukebox,music_live,music_video,music_karaoke
0,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,...,None,None,None,None,None,None,None,None,None,None
1,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,...,None,None,None,None,None,None,None,None,None,None
2,5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,...,None,None,None,None,None,None,None,None,None,None
3,8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,...,None,None,None,None,None,None,None,None,None,None
4,9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,...,None,None,None,None,None,None,None,None,None,None


### Imputing missing values for `hours` feature

In [24]:
# recheck number of missing values
nan_counts = busi_df2.isna().sum()
nan_counts.loc[nan_counts != 0]

hours    9710
dtype: int64

First, we expand operating hour objects into a flat table of 7 days. For businesses whose `hours` is available for some days, we assume that the businesses are closed for the other days that do not have hour information. For businesses whose value for the `hours` column is completely missing, we impute the operating hours with the modes of the dataset. 

In [25]:
# expand and concatenate `hours` to the previous `busi_df_final`
busi_df_final = busi_df2.merge(pd.json_normalize(busi_df2['hours']).add_prefix('hours_'),
                               left_index = True, right_index = True, how = 'left')


for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    # For businesses whose `hours` is available for some days, we assume that the businesses are closed for the other days that do not have hour information.
    busi_df_final.loc[(busi_df_final[f'hours_{day}'].isna()) & (~busi_df_final['hours'].isna()), f'hours_{day}'] = "closed"

    # For businesses whose value for the `hours` column is completely missing, we impute the operating hours with the modes of the dataset.
    busi_df_final[f'hours_{day}'].fillna(busi_df_final[f'hours_{day}'].mode()[0], inplace=True)
    

# drop the original `hours` column
busi_df_final.drop(columns = ['hours'], inplace = True)

busi_df_final.head(3)

,original_index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,...,music_live,music_video,music_karaoke,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday
0,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,...,None,None,None,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
1,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,...,None,None,None,closed,closed,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
2,5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,...,None,None,None,0:0-0:0,6:0-22:0,6:0-22:0,6:0-22:0,9:0-0:0,9:0-22:0,8:0-22:0


### inspecting number of businesses in each state

In [26]:
busi_df_final['state'].value_counts()

PA     16170
FL     11476
TN      5613
IN      5374
MO      5366
LA      4888
NJ      4216
AZ      3579
AB      3048
NV      2462
ID      1798
CA      1695
IL      1198
DE      1171
NC         1
CO         1
HI         1
MT         1
SD         1
XMS        1
Name: state, dtype: int64

Since we are interested in restaurant recommendations, it does not make sense to keep states that have only one business.

In [27]:
busi_df_final = busi_df_final[~busi_df_final['state'].isin(["NC", "CO", "HI", "MT", "SD", "XMS"])]

### Feature Scaling

We will look at what numeric features need to be scaled:

In [29]:
num_cols = busi_df_final.columns[busi_df_final.dtypes.apply(lambda x: x in ['int64', 'float64'])].to_list()
num_cols

['original_index', 'latitude', 'longitude', 'stars', 'review_count', 'is_open']

In [31]:
busi_df_final.loc[:2, num_cols]

,original_index,latitude,longitude,stars,review_count,is_open
0,3,39.955505,-75.155564,4.0,80,1
1,4,40.338183,-75.471659,4.5,13,1
2,5,36.269593,-87.058943,2.0,6,1


Note that `is_open` is binary; `original_index`, `latitude`, `longitude` are more useful at their original state. Hence, we will only scale the remaining two features: `stars` and `review_count`.

In [33]:
# scale
scale_cols = ['stars', 'review_count']

scaler = StandardScaler()
scaled = scaler.fit_transform(busi_df_final.loc[:, scale_cols])

# put scaled values back to original data frame
busi_df_final.drop(scale_cols, axis=1, inplace=True)
busi_df_final = pd.concat([busi_df_final,
                           pd.DataFrame(scaled, columns=scale_cols, index=busi_df_final.index)],
                          axis=1)

In [38]:
busi_df_final.iloc[:2, -2:]

,stars,review_count
0,0.512139,0.031667
1,1.099386,-0.362684


In [39]:
busi_df_final.shape

(68054, 84)

Save cleaned business data to a feather file

In [40]:
busi_df_final.reset_index(drop = True).to_feather("data/yelp_business_cleaned.feather")
print(f"There are {busi_df_final.shape[0]} business records left with {busi_df_final.shape[1]} total features.")

There are 68054 business records left with 84 total features.
